# Q1

参考数据中给出了拱墅区街道划分（示意）`gongshu.geojson`和拱墅区POI数据（示意）`gs_poi.geojson`,

请根据这两个文件，按以下指标对拱墅区各街道的社会宜居性做初略的评价：

- 辖区内小学密度：
    - 辖区内小学的POI数据KIND6属性为`'160102'`


- 综合医院()的覆盖密度：
    - 认为综合医院的覆盖范围为1600米
    - 综合医院的POI数据KIND6属性为`'170101'`

将以上两个指标归一化后直接相加作为社会宜居性得分，并按照得分降序排列给出结果

In [ ]:
import geopandas as gpd
import numpy as np